# Surgery

Move all the weight and biases from MikeNet to TensorFlow

In [1]:
import troubleshooting, meta, modeling, data_wrangling
from importlib import reload
import numpy as np
import tensorflow as tf
reload(troubleshooting)
mn_weight = troubleshooting.MikeNetWeight("Reading_Weight_v1")  # They have v1-v10, looks similar to me

Weight Keys: ['Phono -> psh', 'Con -> csh', 'psh -> Semantics', 'csh -> Semantics', 'Semantics -> SemCleanup', 'SemCleanup -> Semantics', 'Bias -> Semantics', 'Bias -> SemCleanup', 'Bias -> psh', 'Bias -> csh', 'Semantics -> sph', 'sph -> Phono', 'Phono -> PhoCleanup', 'PhoCleanup -> Phono', 'Bias -> Phono', 'Bias -> sph', 'Bias -> PhoCleanup', 'Ortho -> oph', 'Ortho -> osh', 'oph -> Phono', 'osh -> Semantics', 'Bias -> oph', 'Bias -> osh']

Non-weight Keys: ['SimulatorSeed 425840429', 'TAOS Phono', 'DELAYS Phono', 'TAOS psh', 'DELAYS psh', 'TAOS sph', 'DELAYS sph', 'TAOS Semantics', 'DELAYS Semantics', 'TAOS SemCleanup', 'DELAYS SemCleanup', 'TAOS PhoCleanup', 'DELAYS PhoCleanup', 'TAOS Bias', 'DELAYS Bias', 'TAOS Ortho', 'DELAYS Ortho', 'TAOS oph', 'DELAYS oph', 'TAOS osh', 'DELAYS osh', 'TAOS Con', 'DELAYS Con', 'TAOS csh', 'DELAYS csh']


2021-11-10 16:39:25.389212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 16:39:25.399497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 16:39:25.400126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 16:39:25.401110: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

## Shapes of weights in MikeNet model

In [2]:
mn_weight.create_weights_shapes()

{'Phono -> psh': (200, 300),
 'psh -> Semantics': (300, 2446),
 'Semantics -> SemCleanup': (2446, 50),
 'SemCleanup -> Semantics': (50, 2446),
 'Semantics -> sph': (2446, 300),
 'sph -> Phono': (300, 200),
 'Phono -> PhoCleanup': (200, 50),
 'PhoCleanup -> Phono': (50, 200),
 'Ortho -> oph': (364, 500),
 'Ortho -> osh': (364, 500),
 'oph -> Phono': (500, 200),
 'osh -> Semantics': (500, 2446),
 'Bias -> oph': (500,),
 'Bias -> osh': (500,),
 'Bias -> Semantics': (2446,),
 'Bias -> Phono': (200,),
 'Bias -> psh': (300,),
 'Bias -> sph': (300,),
 'Bias -> SemCleanup': (50,),
 'Bias -> PhoCleanup': (50,)}

# Create config file

In [3]:
code_name = "surgery"
batch_name = None
tf_root = "/home/jupyter/triangle_model"

# Model configs
ort_units = 364
pho_units = 200
sem_units = 2446
hidden_os_units = 500
hidden_op_units = 500
hidden_ps_units = 300
hidden_sp_units = 300
pho_cleanup_units = 50
sem_cleanup_units = 50
pho_noise_level = 0.
sem_noise_level = 0.
activation = "sigmoid"

tau = 1 / 3
max_unit_time = 4.0
output_ticks = 13
inject_error_ticks = 11

# Training configs
learning_rate = 0.005
zero_error_radius = 0.1
save_freq = 20

# Environment configs
task_names = ("pho_sem", "sem_pho", "pho_pho", "sem_sem", "ort_pho", "ort_sem", "triangle")
wf_compression = "log"
wf_clip_low = 0
wf_clip_high = 999_999_999
oral_start_pct = 1.0
oral_end_pct = 1.0

total_sample = 1_800_000
tasks_ps = (0.2, 0.2, 0.05, 0.05, .1, .1, .3)

batch_size = 100
rng_seed = 2021

## Build model

In [4]:
cfg = meta.Config.from_dict(**globals())
model = modeling.MyModel(cfg)
model.build()

UUID not found, regenerating.
Saved config json to /triangle_model/models/surgery/model_config.json


# Grafting

In [5]:
for weight in model.weights:
    try:
        name = weight.name[:-2]
        weight.assign(mn_weight.weights_tf[name])
        print(f"Grafted {name} from mikenet to TensorFlow")

        # Post-load weight sanity check
        tf.debugging.assert_equal(mn_weight.weights_tf[name], weight)

    except KeyError:
        print(f"Missing weight {name} in mikenet")
        pass

Grafted w_hsp_sh from mikenet to TensorFlow
Grafted w_hsp_hp from mikenet to TensorFlow
Grafted w_pc from mikenet to TensorFlow
Grafted w_cp from mikenet to TensorFlow
Grafted bias_hsp from mikenet to TensorFlow
Grafted bias_p from mikenet to TensorFlow
Grafted bias_cpp from mikenet to TensorFlow
Grafted w_hps_ph from mikenet to TensorFlow
Grafted w_hps_hs from mikenet to TensorFlow
Grafted w_sc from mikenet to TensorFlow
Grafted w_cs from mikenet to TensorFlow
Grafted bias_hps from mikenet to TensorFlow
Grafted bias_s from mikenet to TensorFlow
Grafted bias_css from mikenet to TensorFlow
Grafted w_hos_oh from mikenet to TensorFlow
Grafted w_hos_hs from mikenet to TensorFlow
Grafted bias_hos from mikenet to TensorFlow
Grafted w_hop_oh from mikenet to TensorFlow
Grafted w_hop_hp from mikenet to TensorFlow
Grafted bias_hop from mikenet to TensorFlow


### Save grafted weights into TF checkpoint format

In [6]:
ckpt = tf.train.Checkpoint(model=model)

ckpt_manager = tf.train.CheckpointManager(
    ckpt,
    cfg.checkpoint_folder,
    max_to_keep=None,  # Keep all checkpoints
    checkpoint_name="epoch",
)

ckpt_manager.save(1)

'/triangle_model/models/surgery/checkpoints/epoch-1'

# Input

In [11]:
import data_wrangling
os.chdir(cfg.tf_root)

In [12]:
mn_r100 = data_wrangling.load_testset('mn_r100')
model.set_active_task('triangle')
pred = model([mn_r100['ort']]* 13)


2021-11-10 16:48:35.974740: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [17]:
from tqdm import tqdm
import os
import tensorflow as tf
import metrics, modeling, gcp
from data_wrangling import load_testset
from helper import get_batch_pronunciations_fast
import pandas as pd
import numpy as np

class Test:
    """Universal test set object for evaluating model results
    1. Single condition, single metric, single value output for maximum compatibility
    2. Model level info should be stored at separate table, and merge it in the end
    """

    def __init__(self, cfg):
        self.cfg = cfg
        self.model = None  # Will create in eval()
        self.ckpt = None  # Will create in eval()
        self.metrics = {
            "acc": {"pho": metrics.PhoAccuracy(), "sem": metrics.RightSideAccuracy()},
            "sse": metrics.SumSquaredError(),
            "act0": metrics.OutputOfZeroTarget(),
            "act1": metrics.OutputOfOneTarget(),
        }

    def eval_train(self, task: str, n: int = 12, to_bq: bool = False):
        """Evaluate the full training set with batching.
        Due to memory demands, will not store df in memory, but save to csv file
        """

        for i in range(n):
            self.eval(f"train_batch_{i}", task, to_bq=to_bq, bq_table='train')

    def eval(
        self,
        testset_name: str,
        task: str,
        save_file_prefix: str = None,
        to_bq: bool = False,
        bq_table: str = None
    ):
        """
        Inputs
        testset_name: name of testset, must match testset package (*.pkl.gz) name
        task: 1 of 9 task option in triangle model
        output: pandas dataframe with all the evaluation results
        """
        try:
            df = self.load(testset_name, task, save_file_prefix)
            print(f"Eval results found, load from saved csv")
        except (FileNotFoundError, IOError):
            assert (bq_table is not None) or (not to_bq)

            df = pd.DataFrame()
            ts_path = "dataset/testsets"
            testset_package = load_testset(
                os.path.join(ts_path, f"{testset_name}.pkl.gz")
            )

            # Enforceing batch_size dim to match with test case
            inputs = testset_package[modeling.IN_OUT[task][0]]

            # Build model and switch task
            self.model = modeling.MyModel(self.cfg, batch_size_override=inputs.shape[0])
            self.ckpt = tf.train.Checkpoint(model=self.model)
            self.model.set_active_task(task)

            # No loading weights, assumes the weights are already loaded
            y_pred = self.model([inputs] * self.cfg.n_timesteps)

            for timetick_idx in range(self.cfg.output_ticks):
                if task == "triangle":
                    for output_name in ("pho", "sem"):
                        df = self._try_to_run_eval(
                            df,
                            y_pred,
                            testset_name,
                            task,
                            1,
                            output_name,
                            timetick_idx,
                            testset_package,
                        )
                else:
                    output_name = modeling.IN_OUT[task][1]
                    df = self._try_to_run_eval(
                        df,
                        y_pred,
                        testset_name,
                        task,
                        1,
                        output_name,
                        timetick_idx,
                        testset_package,
                    )

            # Save evaluation
            if save_file_prefix is not None:
                csv_name = os.path.join(
                    self.cfg.eval_folder,
                    f"{save_file_prefix}_{testset_name}_{task}.csv",
                )
            else:
                csv_name = os.path.join(
                    self.cfg.eval_folder, f"{testset_name}_{task}.csv"
                )

            df.to_csv(csv_name)

            if to_bq:
                gcp.df_to_bigquery(df, self.cfg.batch_name, bq_table)
        return df

    def _try_to_run_eval(
        self,
        df,
        y_pred,
        testset_name,
        task,
        epoch,
        output_name,
        timetick_idx,
        testset_package,
    ):

        if testset_package[output_name] is not None:

            tag = {
                "code_name": self.cfg.code_name,
                "epoch": epoch,
                "testset": testset_name,
                "task": task,
                "output_name": output_name,
                "timetick_idx": timetick_idx,
                "timetick": self.output_idx_to_timetick(timetick_idx),
                "word": testset_package["item"],
                "cond": testset_package["cond"],
            }

            df = df.append(
                self._eval_one(y_pred, testset_package, tag), ignore_index=True
            )

        return df

    def load(self, testset_name, task, save_file_prefix=None):
        if save_file_prefix is not None:
            csv_file = os.path.join(
                self.cfg.eval_folder, f"{save_file_prefix}_{testset_name}_{task}.csv"
            )
        else:
            csv_file = os.path.join(self.cfg.eval_folder, f"{testset_name}_{task}.csv")
        return pd.read_csv(csv_file, index_col=0)

    def output_idx_to_timetick(self, idx):
        # Zero indexing idx to one indexing step
        d = self.cfg.n_timesteps - self.cfg.output_ticks
        return idx + 1 + d

    def _eval_one(self, y_pred, y_true, tag):
        """
        y_pred: predition dictionary, e.g., {'pho': (time ticks, items, output nodes)}
        y_true: label dictionary (time invarying), e.g., {'sem': (items, maybe n ans. output nodes)}
        """
        out = pd.DataFrame()

        this_y_pred = y_pred[tag["output_name"]][tag["timetick_idx"]]
        # shape: (time ticks, items, output nodes)

        this_y_true = y_true[tag["output_name"]]

        # shape: (item, *maybe n ans, output nodes)

        acc = self.metrics["acc"][tag["output_name"]]
        sse = self.metrics["sse"]
        act0 = self.metrics["act0"]
        act1 = self.metrics["act1"]

        if type(this_y_true) is list:
            # List mode (for Glushko)
            out["acc"] = acc.item_metric_multi_list(this_y_true_phoneme, this_y_pred)
            out["sse"] = sse.item_metric_multi_list(this_y_true, this_y_pred)
            # TODO: add act0 and act1
        elif tf.rank(this_y_true) == 3:
            # Multi ans mode if we have 3 dims
            out["acc"] = acc.item_metric_multi_ans(this_y_true, this_y_pred)
            out["sse"] = sse.item_metric_multi_ans(this_y_true, this_y_pred)
            # TODO: add act0 and act1
        else:
            # Single ans mode
            out["acc"] = acc.item_metric(this_y_true, this_y_pred)
            out["sse"] = sse.item_metric(this_y_true, this_y_pred)
            out["act0"] = act0.item_metric(this_y_true, this_y_pred)
            out["act1"] = act1.item_metric(this_y_true, this_y_pred)

        # Write prediction if output is pho
        if tag["output_name"] == "pho":
            out["pho_pred"] = get_batch_pronunciations_fast(this_y_pred)

        # Write tag to df
        for k, v in tag.items():
            out[k] = v

        return out



In [19]:
test = Test(cfg)

In [20]:
test.eval('mn_r100', 'triangle')

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:189 f  *
        iters,
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:286 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:527 loop_fn
        return fn(gathered_elems)
    /triangle_model/src/metrics.py:189 get_pho_idx_item
        act_2d = tf.reshape(tf.cast(act, tf.float32), shape=(10, 25))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:196 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_array_ops.py:8404 reshape
        "Reshape", tensor=tensor, shape=shape,

In [18]:

d = troubleshooting.Diagnosis(code_name)
d.eval('mn_r100', task='triangle', epoch=1)

@interact(
    sel_word=d.testset_package['item'], 
    layer=['pho', 'sem'], 
    task=['triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(10, d.cfg.total_number_of_epoch + 1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d = troubleshooting.Diagnosis(code_name)
    d.eval(testset_name, task=task, epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer(layer)


Loaded config from models/surgery/model_config.json


Evaluating mn_r100:triangle:   0%|          | 0/31 [00:00<?, ?it/s]

Cannot find phoneme in y_true dictionary


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:189 f  *
        iters,
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:286 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:527 loop_fn
        return fn(gathered_elems)
    /triangle_model/src/metrics.py:189 get_pho_idx_item
        act_2d = tf.reshape(tf.cast(act, tf.float32), shape=(10, 25))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:196 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_array_ops.py:8404 reshape
        "Reshape", tensor=tensor, shape=shape,